In [1]:
# MLP Model w 2 Hidden Layers, dropout rate = 0.01, learning rate = 0.01, Adam optimizer
# Input layer --> 2 Hidden Layers (each w Linear, ReLU, Dropout) --> Output layer

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import random

# Set random seed for reproducibility
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)

# Load the data
merged_downtown_data = pd.read_csv('./merged_downtown_data.csv')
ridesourcing_data = pd.read_csv("./Ridesourcing_CensusCount_ALL_0_Filled.csv")

# Renaming and setting index for merging
merged_downtown_data = merged_downtown_data.rename(columns={"TractID": "index"})
merged_downtown_data.set_index("index", inplace=True)
ridesourcing_data.set_index("index", inplace=True)

# Merging the dataframes
merged_df = merged_downtown_data.join(ridesourcing_data, how='inner')

# Dropping the unnecessary columns
columns_to_drop = ["Unnamed: 0", "X", "Y"]
merged_df = merged_df.drop(columns=columns_to_drop)

# Create the "is_downtown" column
downtown_areas = [
    '17031839000', '17031080202', '17031833000', '17031833100', '17031839100', 
    '17031081201', '17031080201', '17031081202', '17031842200', '17031838300', 
    '17031081401', '17031081403', '17031081300', '17031081100', '17031080300', 
    '17031080100', '17031080400', '17031081000', '17031320400', '17031320600', 
    '17031081402', '17031320100', '17031081900', '17031081500', '17031081800', 
    '17031081600', '17031081700', '17031280100', '17031281900'
]

downtown_areas = [int(tract_id) for tract_id in downtown_areas]
merged_df['is_downtown'] = merged_df.index.isin(downtown_areas).astype(int)

# Verify the 'is_downtown' column
print("Is Downtown Column Distribution:\n", merged_df['is_downtown'].value_counts())

# Aggregate the travel demand by summing across all time columns
ridesourcing_data_aggregated = ridesourcing_data.sum(axis=1)

# Splitting the data into features and aggregated target
features = merged_df.drop(columns=ridesourcing_data.columns)
target = ridesourcing_data_aggregated

# Create MinMaxScaler instances for features and target
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

# Fit and transform the features and target
features_scaled = feature_scaler.fit_transform(features)
target_scaled = target_scaler.fit_transform(target.values.reshape(-1, 1)).flatten()

# Use stratified sampling to split the data
train_indices, test_indices = train_test_split(range(len(features_scaled)), test_size=0.2, stratify=merged_df['is_downtown'])
X_train, X_test = features_scaled[train_indices], features_scaled[test_indices]
y_train, y_test = target_scaled[train_indices], target_scaled[test_indices]
is_downtown_train = merged_df['is_downtown'].iloc[train_indices].to_numpy().astype(float)
is_downtown_test = merged_df['is_downtown'].iloc[test_indices].to_numpy().astype(float)

# Define the PyTorch dataset and dataloader
class TravelDataset(Dataset):
    def __init__(self, X, y, is_downtown):
        self.X = X
        self.y = y
        self.is_downtown = is_downtown

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx, :].astype(np.float32)
        y = np.array(self.y[idx], dtype=np.float32)
        is_downtown = np.array(self.is_downtown[idx], dtype=np.float32)
        return torch.from_numpy(x), torch.from_numpy(y), torch.from_numpy(is_downtown)

train_dataset = TravelDataset(X_train, y_train, is_downtown_train)
test_dataset = TravelDataset(X_test, y_test, is_downtown_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the Net model
class Net(nn.Module):
    def __init__(self, n_features, n_neurons, device, seq_len, pre_len):
        super(Net, self).__init__()
        self.flatten = nn.Flatten()
        self.device = device
        self.n_features = n_features
        self.n_neurons = n_neurons
        self.seq_len = seq_len
        self.pre_len = pre_len
        
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(n_features, n_neurons),
            nn.ReLU(),
            nn.Dropout(p=0.01),
            nn.Linear(n_neurons, n_neurons),
            nn.ReLU(),
            nn.Dropout(p=0.01),
            nn.Linear(n_neurons, pre_len),
            nn.ReLU(),
            nn.Dropout(p=0.01),
        )

    def forward(self, x):
        pred = self.linear_relu_stack(x).to(self.device)
        return pred.squeeze()

def combined_loss(outputs, targets, is_downtown, lam=0.5):
    mse_loss = nn.MSELoss()(outputs, targets)
    monotonicity_penalty = 0.0
    violation_count = 0

    # Efficiently compute monotonicity penalty
    for i in range(len(outputs)):
        for j in range(len(outputs)):
            if is_downtown[i] > is_downtown[j] and outputs[i] < outputs[j]:
                monotonicity_penalty += (outputs[j] - outputs[i]).item()
                violation_count += 1

    cost = mse_loss + lam * monotonicity_penalty
    return cost, mse_loss, monotonicity_penalty, violation_count

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 100
lam_values = [0.0, 0.2, 0.5, 0.8, 1.0]
results = {lam: {'train_violation_count': 0, 'test_violation_count': 0, 'mape': 0, 'mse': 0, 'mae': 0, 'rmse': 0} for lam in lam_values}

for lam in lam_values:
    print(f"Training with lam = {lam}")
    
    model = Net(n_features=features.shape[1], n_neurons=256, device=device, seq_len=1, pre_len=1)
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    criterion = lambda outputs, targets, is_downtown: combined_loss(outputs, targets, is_downtown, lam)
    
    best_val_loss = float('inf')
    best_model_state_dict = None
    total_train_violation_count = 0
    total_test_violation_count = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_mse_loss = 0.0
        train_monotonicity_penalty = 0.0
        train_violation_count = 0
        for inputs, targets, is_downtown_batch in train_loader:
            inputs, targets, is_downtown_batch = inputs.to(device), targets.to(device), is_downtown_batch.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss, mse_loss, monotonicity_penalty, violation_count = criterion(outputs, targets, is_downtown_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_mse_loss += mse_loss.item()
            train_monotonicity_penalty += monotonicity_penalty
            train_violation_count += violation_count

        train_loss /= len(train_loader)
        train_mse_loss /= len(train_loader)
        train_monotonicity_penalty /= len(train_loader)
        train_violation_count /= len(train_loader)
        total_train_violation_count += train_violation_count

        model.eval()
        with torch.no_grad():
            val_loss = 0.0
            val_mse_loss = 0.0
            val_monotonicity_penalty = 0.0
            val_violation_count = 0
            for inputs, targets, is_downtown_batch in test_loader:
                inputs, targets, is_downtown_batch = inputs.to(device), targets.to(device), is_downtown_batch.to(device)
                outputs = model(inputs)
                loss, mse_loss, monotonicity_penalty, violation_count = criterion(outputs, targets, is_downtown_batch)
                val_loss += loss.item()
                val_mse_loss += mse_loss.item()
                val_monotonicity_penalty += monotonicity_penalty
                val_violation_count += violation_count

            val_loss /= len(test_loader)
            val_mse_loss /= len(test_loader)
            val_monotonicity_penalty /= len(test_loader)
            val_violation_count /= len(test_loader)
            total_test_violation_count += val_violation_count

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model_state_dict = model.state_dict()

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train MSE Loss: {train_mse_loss:.4f}, Train Monotonicity Penalty: {train_monotonicity_penalty:.4f}, Train Violation Count: {train_violation_count:.2f}, Val Loss: {val_loss:.4f}, Val MSE Loss: {val_mse_loss:.4f}, Val Monotonicity Penalty: {val_monotonicity_penalty:.4f}, Val Violation Count: {val_violation_count:.2f}')
      
    # Load the best model based on the validation loss
    model.load_state_dict(best_model_state_dict)
    
    # Evaluate the best model on the test set
    model.eval()
    with torch.no_grad():
        y_true = []
        y_pred = []
        for inputs, targets, _ in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            y_true.extend(targets.cpu().numpy())
            y_pred.extend(outputs.cpu().numpy())
    
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_true = target_scaler.inverse_transform(y_true.reshape(-1, 1)).flatten()
    y_pred = target_scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()
    mape = mean_absolute_percentage_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    
    results[lam]['train_violation_count'] = total_train_violation_count / num_epochs
    results[lam]['test_violation_count'] = total_test_violation_count / num_epochs
    results[lam]['mape'] = mape
    results[lam]['mse'] = mse
    results[lam]['mae'] = mae
    results[lam]['rmse'] = rmse
    
    print(f"lam = {lam} - MAPE: {mape:.4f}, MSE: {mse:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}")

best_lam = min(results, key=lambda x: results[x]['mape'])
print(f"Best lam: {best_lam}")
print(f"Best Model - MAPE: {results[best_lam]['mape']:.4f}, MSE: {results[best_lam]['mse']:.4f}, MAE: {results[best_lam]['mae']:.4f}, RMSE: {results[best_lam]['rmse']:.4f}")
print(f"Average Train Violation Count: {results[best_lam]['train_violation_count']:.2f}")
print(f"Average Test Violation Count: {results[best_lam]['test_violation_count']:.2f}")

Is Downtown Column Distribution:
 is_downtown
0    742
1     29
Name: count, dtype: int64
Training with lam = 0.0
Epoch [1/100], Train Loss: 0.0038, Train MSE Loss: 0.0038, Train Monotonicity Penalty: 0.4718, Train Violation Count: 17.80, Val Loss: 0.0011, Val MSE Loss: 0.0011, Val Monotonicity Penalty: 0.0003, Val Violation Count: 0.33
Epoch [2/100], Train Loss: 0.0018, Train MSE Loss: 0.0018, Train Monotonicity Penalty: 0.0000, Train Violation Count: 0.00, Val Loss: 0.0023, Val MSE Loss: 0.0023, Val Monotonicity Penalty: 0.0000, Val Violation Count: 0.00
Epoch [3/100], Train Loss: 0.0019, Train MSE Loss: 0.0019, Train Monotonicity Penalty: 0.0077, Train Violation Count: 0.20, Val Loss: 0.0019, Val MSE Loss: 0.0019, Val Monotonicity Penalty: 0.0000, Val Violation Count: 0.00
Epoch [4/100], Train Loss: 0.0012, Train MSE Loss: 0.0012, Train Monotonicity Penalty: 0.0003, Train Violation Count: 0.10, Val Loss: 0.0018, Val MSE Loss: 0.0018, Val Monotonicity Penalty: 0.0007, Val Violation C